In [ ]:
# !pip install git+https://github.com/EleutherAI/lm-evaluation-harness.git
# !pip install tiktoken protobuf transformers accelerate sentencepiece bitsandbytes
from transformers import AutoModel, AutoTokenizer, BitsAndBytesConfig

In [ ]:
# from huggingface_hub import login
# 

In [ ]:
# model_name          = "meta-llama/Llama-2-7b-hf"
# model_name          = "Rabinovich/wanda_pruned_llama_2_7B" # 50 %
# model_name          = "Arjun7m/Wanda-pruned-Llama-2-7b-0.3" # 30%

# quantization_config = BitsAndBytesConfig(load_in_8bit=True)

# 4-bit Quantization
# Update: BitsAndBytesConfig(load_in_4bit=True, bnb_4bit_compute_dtype=torch.float16) --> Check
## Also check: bnb_4bit_quant_type="nf4", bnb_4bit_use_double_quant=True
# quantization_config = BitsAndBytesConfig(load_in_4bit=True)

# model               = AutoModel.from_pretrained(model_name, trust_remote_code=True, quantization_config=quantization_config)

model                 = AutoModel.from_pretrained(model_name, trust_remote_code=True)
tokenizer             = AutoTokenizer.from_pretrained(model_name)

In [ ]:
save_path = "./magnitude_pruned_llama_2_7B"
model.save_pretrained(save_path)
tokenizer.save_pretrained(save_path)

In [ ]:
!lm-eval --model hf \
         --model_args pretrained=./magnitude_pruned_llama_2_7B \
         --tasks piqa,winogrande \  
         --device cuda \
        #  --limit 10
        #  --output output/hellaswag/ \
        #  --log_samples


# Tasks - acp_bench,anli,commonsense_qa,mmlu,piqa,pubmedqa,winogender

# accuracy - boolq (2), piqa (1), hellaswag (10), winogrande (1)
# perplexity - wikitext (1)